## Imports

In [ ]:
import torch
import torch.nn as nn # All neural network modules, nn.Linear, nn.Conv2D, BatchNorm, Loss Functions
import torch.optim as optim # All optimization algorithms, SGD, Adam, etc
from torch.optim import lr_scheduler
import torch.nn.functional as F
import torchvision.datasets as datasets # Has standard datasets we can import in a nice way
import torchvision.transforms as transforms # Transformation we can perform on our dataset
from torch.utils.data import DataLoader # Gives easier dataset management and create mini batches
from torch.utils.tensorboard import SummaryWriter # To print to Tensorboard
from torchvision import models
import torchvision

# Utils
import time
import copy
import re
import cv2
from pathlib import Path
from sklearn.utils import shuffle
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline



In [ ]:
# Patch for (https://github.com/pytorch/tutorials/issues/762)
# for Tensorboard projector T-SNE, PCA
import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

## Create Fully Connected Network

## Set Device

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

## Hyperparameters

In [ ]:
learning_rate = 1e-3
in_channels = 1
num_classes = 10
batch_size = 4
num_epochs = 1
idx_to_class = ['0','1','2','3','4','5','6','7','8','9']

## Data Process

In [ ]:
train_dataset = ...
train_loader = ...

test_dataset = ...
test_loader = ...

## Model and Optimizer

In [ ]:
model = ...
model.to(device)

In [ ]:
criterion = ...
optimizer = ...

### CLI setup

Make sure run Tensorboard in this directory
```bash
$ docker exec -it ml_lab1 bash
$ conda activate tf2
$ cd D/Corona/MaskedFaceDetection/RMFD
$ tensorboard --logdir=runs
```

## Predefined utility functions

In [ ]:
def show(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)), interpolation='nearest')

In [ ]:
def save(model, optimizer, loss, filename):
    save_dict = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss.item()
        }
    torch.save(save_dict, filename)

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=2):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    step = 0

    for epoch in range(num_epochs):
        losses = []
        accuracies = []
        
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
#         for phase in ['train', 'val']:
        for phase in ['train']:
            if phase == 'train':
                dataset_size = dataset_sizes[0]
                dataloader = train_loader
                model.train()  # Set model to training mode
            else:
                dataset_size = dataset_sizes[1]
                dataloader = test_loader
                model.eval()   # Set model to evaluate mode

            # Iterate over data.
            for batch_idx, (inputs, labels) in enumerate(dataloader):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                # statistics
                num_corrects = torch.sum(preds == labels.data)
                running_train_acc = float(num_corrects)/float(inputs.shape[0])
                accuracies.append(running_train_acc)
                losses.append(loss.item())
                
                # Tensorboard Running
                ## Data
                img_grid = torchvision.utils.make_grid(inputs)
                writer.add_image('mnist_images', img_grid)
                
                ## Check how the weight changes in specific layer
                writer.add_histogram('fc1', model.fc1.weight)
                
                ## Metrics
                writer.add_scalar('Training Loss', loss.item(), global_step=step)
                writer.add_scalar('Training Accuracy', running_train_acc, global_step=step)
                step += 1
                
                ## Check how the model predicts eg. PCA, T-SNE
                features = inputs.reshape(inputs.shape[0], -1)
                class_labels = [idx_to_class[label] for label in preds]
                if batch_idx == 230:
                    writer.add_embedding(features, metadata=class_labels, label_img=inputs, global_step=batch_idx)

            if phase == 'train' and scheduler != None:
                scheduler.step()

            # Tensorboard Epoch
            writer.add_hparams({'lr': learning_rate, 'bsize': batch_size},
                       {'accuracy': sum(accuracies)/len(accuracies), 
                       'loss': sum(losses)/len(losses)})
    
            epoch_loss = sum(losses)/len(losses)
            epoch_acc = sum(accuracies)/len(accuracies)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    
    # save model
#     save(model, optimizer, loss, 'resnet18-mask-saved.pth')
    
    return model

## Hyper Parameter search

In [ ]:
# Num of parameters to train
sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
# Test Dataloader and Model
train_iter = iter(train_loader)
samples , labels = train_iter.next()
print(samples.shape, labels.shape)
print(labels[:6])
img_grid = torchvision.utils.make_grid(samples)
show(img_grid)

print(model(samples.to(device)))

In [ ]:
batch_sizes = [256]
learning_rates = [0.01, 0.001]
dataset_sizes = [len(train_dataset), len(test_dataset)]

# scheduler
step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
for batch_size in batch_sizes:
    for learning_rate in learning_rates:
        # Reset Model
        model = CNN(in_channels=in_channels, num_classes=num_classes)
        model.to(device)
        # Dataset and Optimizer according to hyperparameter
        train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.0)
        # Tensorboard file
        writer = SummaryWriter(f'runs/<Project>/MiniBatchSize {batch_size} LR {learning_rate}')
        train_model(model, criterion, optimizer, None, num_epochs)

## Deploy

In [ ]:
load_checkpoint = torch.load('resnet18-mask-saved.pth')

In [ ]:
resnet = models.resnet18(pretrained=True)
num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, 2)

resnet.load_state_dict(load_checkpoint['model_state_dict'])
resnet.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(resnet.parameters(), lr=0)

optimizer.load_state_dict(load_checkpoint['optimizer_state_dict'])

In [ ]:
# scheduler
step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
resnet_trained = train_model(resnet, criterion, optimizer, step_lr_scheduler, num_epochs=2)

## Inference

In [ ]:
test_compose = transforms.Compose([transforms.ToPILImage(),
                                   transforms.Resize(224),
                                   transforms.ToTensor(),
                                  ])

In [ ]:
def predict(model, numpy_image):
    plt.imshow(numpy_image)
     
    test_image_tensor = test_compose(numpy_image)
    test_image_tensor = test_image_tensor.unsqueeze_(0).to(device)
     
    with torch.no_grad():
        model.eval()
        # Model outputs log probabilities
        out = model(test_image_tensor)
        ps = torch.exp(out)
        topk, topclass = ps.topk(1, dim=1)
        print("Output class :  ", idx_to_class[topclass.cpu().numpy()[0][0]])
        

In [ ]:
predict(resnet_trained, test_dataset[47,0])